In [63]:
#Ram Ram
#import networkx as nx
import random
from shapely.geometry import Point, Polygon
import numpy as np
import random
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np

In [64]:
max_X=200
max_Y=350
min_X=0
min_Y=0
step_length_for_divide_X=10
step_length_for_divide_Y=10
intersections=[]
geometric_centers=[]
for i in range(0,max_X+step_length_for_divide_X,step_length_for_divide_X):
    for j in range(0,max_Y+step_length_for_divide_Y,step_length_for_divide_Y):
        intersections.append(tuple((i,j)))
        #print((i,j))

In [65]:
xcoords=[]
ycoords=[]
for i in range(len(intersections)):
    xcoords.append(intersections[i][0])
    ycoords.append(intersections[i][1])
# plt.figure(figsize=(20,20))
# plt.scatter(xcoords,ycoords)
# print(intersections)

In [66]:
geometric_centers=[]
for item1 in intersections:
    for item2 in intersections:
        if item1 != item2:
            x1,y1=item1[0],item1[1]
            x2,y2=item2[0],item2[1]
            if abs(x2-x1)==step_length_for_divide_X and abs(y2-y1)==step_length_for_divide_Y:
                geometric_centers.append(tuple(((x1+x2)/2,(y1+y2)/2)))
geometric_centers=[*set(geometric_centers)]
# xcoords_centers=[]
# ycoords_centers=[]
# plt.figure(figsize=(20,20))
# for i in range(len(geometric_centers)):
#     xcoords_centers.append(geometric_centers[i][0])
#     ycoords_centers.append(geometric_centers[i][1])
# plt.scatter(xcoords,ycoords,color='blue')
# plt.scatter(xcoords_centers,ycoords_centers,color='red')
# print(geometric_centers)

In [67]:
#Without defining the power load of each area we are going to define the EV demand per fast charging demand point
# EV_demand={}
# min_EV_load=1
# max_EV_load=10
# for point in geometric_centers:
#     EV_demand[point]=random.randint(min_EV_load,max_EV_load)
#print(EV_demand)

In [68]:
#Defining EV Demand Via Text File
EV_demand={}
with open(r'C:\Users\HARIHARAN SUBRAMANIA\Desktop\IPSO Charging Station Demand.txt') as f:
    text_charging_demand = f.read()
charging_values=text_charging_demand.split("\n")
for value in charging_values:
    v=value.split(" ")
    if "" in v:
        v.remove("")
    if "}" == v[2][-1]:
        v[2]=v[2][0:len(v[2])-1]
    dem=int(v[2].split(",")[0])
    origin=int(v[0].split("(")[1].split(",")[0].split(".")[0])
    dest=int(v[1].split(")")[0].split(".")[0])
    EV_demand[(origin,dest)]=dem

In [69]:
# plt.figure(figsize=(20,20))
# plt.scatter(xcoords_centers,ycoords_centers,color='red')
# for k,v in EV_demand.items():
#     plt.text(k[0],k[1],str(v),size=20)

In [70]:
#Formatting cell.....Nothing Imp
geometric_centers_list=[]
for point in geometric_centers:
    geometric_centers_list.append([float(point[0]),float(point[1])])

In [71]:
# nc=50
# charging_station_coordinates=[]
# for i in range(30):
#     x=random.randint(1,200)
#     y=random.randint(1,300)
#     charging_station_coordinates.append(tuple((x,y)))
# charging_station_coordinates_list=[]
# for point in charging_station_coordinates:
#     charging_station_coordinates_list.append([float(point[0]),float(point[1])])
# # charging_station_coordinates_list.append([float(0),float(0)])
# # charging_station_coordinates_list.append([float(max_X),float(0)])
# # charging_station_coordinates_list.append([float(0),float(max_Y)])
# # charging_station_coordinates_list.append([float(max_X),float(max_Y)])
# points = np.array(charging_station_coordinates_list)
# vor = Voronoi(points)
# fig = plt.figure()
# fig = voronoi_plot_2d(vor)
# fig.set_figwidth(20)
# fig.set_figheight(10)
# plt.show()

In [72]:
def region_to_vertex_mapper(regions,vertices):   
    region_to_vertex_mapping={}
    index=0
    new_regions=[]
    for region in regions:
        if len(region)!=0:
            new_regions.append(region)
    regions=new_regions.copy()
    for region in regions:
        region_to_vertex_mapping[index]=[]
        for vertex_index in region:
            if vertex_index!=-1 and vertex_index in range(0,len(vertices)):
                region_to_vertex_mapping[index].append(tuple((vertices[vertex_index][0],vertices[vertex_index][1])))
        if len(region_to_vertex_mapping)==0:
            del region_to_vertex_mapping[index]
            index=index
        else:
            index+=1    
    return region_to_vertex_mapping

In [73]:
#check if point is in triangle or not
def area(x1, y1, x2, y2, x3, y3):
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1)+ x3 * (y1 - y2)) / 2.0)
def isInside(x1, y1, x2, y2, x3, y3, x, y):
    A = area (x1, y1, x2, y2, x3, y3)
    A1 = area (x, y, x2, y2, x3, y3)
    A2 = area (x1, y1, x, y, x3, y3)
    A3 = area (x1, y1, x2, y2, x, y)
    if(A == A1 + A2 + A3):
        return True
    else:
        return False

In [74]:
def get_point_belonging_dictionary(points,polygon_coordinates_dict):
    
    set_of_polygons={}
    index=0
    #print(polygon_coordinates_dict)
    for key in polygon_coordinates_dict.keys():
        #print(polygon_coordinate)
        if len(polygon_coordinates_dict[key])>3:
            set_of_polygons[index]=Polygon(polygon_coordinates_dict[key])
        else:
            set_of_polygons[index]=polygon_coordinates_dict[key]
        index+=1
    #print(set_of_polygons.keys(),polygon_coordinates_dict.keys())
    #print(set_of_polygons)
    point_region_dictionary={}
    for point in points:
        point_region_dictionary[tuple((point[0],point[1]))]=[]
    ctr=0
    for point in points:
        #print(point)
        index=0
        for key in set_of_polygons.keys():
            #print(np.array(polygon_coordinates_dict[key]))
            if len(np.array(polygon_coordinates_dict[key]))>3:
                p=Point(float(point[0]),float(point[1]))
                #print(set_of_polygons[key])
                p.within(set_of_polygons[key])
                if set_of_polygons[key].contains(p)==True:
                    ctr+=1
                    #print(set_of_polygons[key].contains(p))
                if set_of_polygons[key].contains(p):
                    point_region_dictionary[tuple((point[0],point[1]))]=[polygon_coordinates_dict[key],index]
            elif len(np.array(polygon_coordinates_dict[key]))==3:
                if isInside(polygon_coordinates_dict[key][0][0],polygon_coordinates_dict[key][0][1],polygon_coordinates_dict[key][1][0],polygon_coordinates_dict[key][1][1],polygon_coordinates_dict[key][2][0],polygon_coordinates_dict[key][2][1],point[0],point[1]):
                    point_region_dictionary[tuple((point[0],point[1]))]=[polygon_coordinates_dict[key],index]
#             elif len(np.array(polygon_coordinates_dict[key]))<3:
#                 point_region_dictionary[tuple((point[0],point[1]))]=[polygon_coordinates_dict[key],index]
            index+=1
    #print(ctr)
    return point_region_dictionary

#point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list,region_to_vertex_mapping)



In [75]:
def charging_station_to_region_no_mapper(vor):
    charging_station_to_region_no_mapping={}
    cutter=0
    for i in range(len(vor.regions)):
        if len(vor.regions[i])==0:
            cutter=i
            break
    for i in range(len(vor.points)):
        if vor.point_region[i]<=cutter:
            charging_station_to_region_no_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]
        else:
            charging_station_to_region_no_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]-1
    return charging_station_to_region_no_mapping
        
    

In [76]:
def get_nearest_charging_station(point,vor,charging_station_to_region_no_mapping):
    charging_station_to_vertex_mapping=charging_station_vertex_mapper(vor)
    min_dist_dict={}
    curr_dist=10000000000000.0
    curr_key=-1
    for key in charging_station_to_region_no_mapping.keys():
        temp=np.sqrt(((key[0]-point[0])**2)+((key[1]-point[1])**2))
        if temp<=curr_dist:
            curr_dist=temp
            curr_coord=key
    return charging_station_to_vertex_mapping[curr_coord]

In [77]:
def charging_station_vertex_mapper(vor):
    charging_station_to_vertex_mapping={}
    cutter=0
    for i in range(len(vor.regions)):
        if len(vor.regions[i])==0:
            cutter=i
            break
    for i in range(len(vor.points)):
        if vor.point_region[i]<=cutter:
            charging_station_to_vertex_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]
        else:
            charging_station_to_vertex_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]-1
    return charging_station_to_vertex_mapping

        
def region_no_to_charging_station_mapper(vor, charging_station_to_region_no_mapping, point_region_dictionary, region_to_vertex_mapping):
    region_no_to_charging_station_mapping={}
    for i in range(len(vor.regions)-1):
        for point in vor.points:
            if charging_station_to_region_no_mapping[tuple((point[0],point[1]))]==i:
                region_no_to_charging_station_mapping[i]=tuple((point[0],point[1]))

    for key in point_region_dictionary.keys():
        if len(point_region_dictionary[key])==0:
            region_no=get_nearest_charging_station(key,vor,charging_station_to_region_no_mapping)
            point_region_dictionary[key]=[region_to_vertex_mapping[region_no],region_no,region_no_to_charging_station_mapping[region_no]]
        else:
            point_region_dictionary[key]=[point_region_dictionary[key][0],point_region_dictionary[key][1],region_no_to_charging_station_mapping[point_region_dictionary[key][1]]]
    return region_no_to_charging_station_mapping

In [78]:
def get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list):
    total_demand={}
    for station in charging_station_coordinates_list:
        temp_demand=0
        
        for key in point_region_dictionary.keys():
            if point_region_dictionary[key][2]==tuple((station[0],station[1])) or point_region_dictionary[key][2]==tuple((station[1],station[0])):
                temp_demand+=EV_demand[key]
        total_demand[tuple((station[0],station[1]))]=temp_demand
    return total_demand
#total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)

    

In [79]:
def demand_point_to_station_mapper(point_region_dictionary):
    demand_to_charging_station_mapper={}
    for key in point_region_dictionary.keys():
        demand_to_charging_station_mapper[key]=point_region_dictionary[key][2]
    return demand_to_charging_station_mapper
#demand_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)

def station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list):
    station_to_demand_mapper_dict={}
    for station in charging_station_coordinates_list:
        station_to_demand_mapper_dict[tuple((station[0],station[1]))]=[]
    for key in point_region_dictionary.keys():
        try:
            station_to_demand_mapper_dict[point_region_dictionary[key][2]].append(key)
        except:
            station_to_demand_mapper_dict[tuple((point_region_dictionary[key][2][1],point_region_dictionary[key][2][0]))].append(key)
    return station_to_demand_mapper_dict
#station_to_demand_mapper_dict=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)


In [80]:
# The Split EV demand w.r.t service and Total EV demand has been verified
# elec_list=[]
# for key in point_region_dictionary.keys():
#     if point_region_dictionary[key][2] not in elec_list:
#         elec_list.append(point_region_dictionary[key][2])
# sum(total_demand.values()), sum(EV_demand.values())

##### No of Chargers in a Charging Station


$
N_i=\left[\frac{\mu_i \sum_{j \in J_{C N i}} n_j}{n_S}\right] i \in J_{C S}
$

$
$$\newline N_i \text{- No of Chargers in a Charging station i}$$
$$\newline \mu_i \text{- Probability of EV users arriving at a charging station i at the same time }$$
$$\newline n_s \text{- No of EVs in a queue that can be accepted by EV users}$$
$$\newline J_{CN_i} \text{- Collection of all fast charging demand points located in the service area i}$$
$$\newline n_j \text{- No of EV car demand in fast charging demand point j}$$
$

##### Get No of Chargers Function

In [81]:
def get_no_of_chargers( demand_by_service_area ,mu=0.6, max_users=3):
    no_of_chargers={}
    for k in demand_by_service_area.keys():
        no_of_chargers[k]=int((mu*demand_by_service_area[k])/max_users)
    #print(no_of_chargers)
    return no_of_chargers

### Objective Function and Constraints

$ \min F_{\cos t}=F_1+F_2+F_3 $
$ \newline F_1=\sum_{i \in J_{C S}}\left[f_{C S}\left(N_i\right) \frac{r_0\left(1+r_0\right)^z}{\left(1+r_0\right)^z-1}+u_{C S}\left(N_i\right)\right] $
$ \newline F_2=365 a p \sum_{i \in J_{C S}} \sum_{j \in J_{C N_i}} n_j \lambda_{i j} d_{i j} $

$ \newline f_{C S}\left(N_i\right)=W+q N_i+e N_i^2$
$ \newline N_{i \cdot \min } \leq N_i \leq N_{i \cdot \max } i \in J_{C S} $
$ \newline \lambda_{i j} d_{i j} \leq d_{\max } i \in J_{C S}, j \in J_{C N i} $
$ \newline \lambda_{i j} D_{i j} \geq D_{\min } i, j \in J_{C S} ; i \neq j $

### Where:

$ F_{cost} \text{- Annual Social cost of the charging station} $ 
$ \newline F_{1} \text{- Annual Construction and Operation Cost of the Charging Station} $
$ \newline F_{2} \text{- Annual Loss cost of EV users on the way to the Charging Station} $
$ \newline F_{3} \text{- Annual Network Loss Cost of the Powergrid} $
$ \newline J_{CS} \text{-Set of Charging Stations} $
$ \newline f_{CS}(N_i) \text{- Construction Cost Function of Charging Station i} $
$ \newline N_{i} \text{- No of Chargers in Charging Station i} $
$ \newline r_{0} \text{- Diccount Rate} $
$ \newline z \text{- Depreciation Rate} $
$ \newline u_{CS}{(N_i)} \text{- Operating Cost function of Charging Station i valued as certain percentage of construction cost} $
$ \newline W \text{-Investment cost which includes initial construction cost and land purchase cost } $
$ \newline q \text{- Price of Charger} $
$ \newline e \text{- Auxiliary Investment Coefficient} $
$ \newline a \text{- Amt of Electricity Consumed by EV to travel 1 km} $
$ \newline p \text{- Charging Price of EV} $
$ \newline J_{CN_{i}} \text{- Collection of Fast Charging Demand points in service area of charging station i} $
$ \newline \lambda_{ij} \text{- ZigZag Coefficient of Road between charging station i and fast charging demand point j} $
$ \newline d_{ij} \text{- Linear distance space between charging station i and fast charging demand point j} $
$ \newline T \text{- Daily Work Time of Charging Station} $
$ \newline M_i \text{- No. of Transformers in charging station i} $
$ \newline C_{Cu},C_{Fe} \text{- Copper and Iron Loss converted to Converted to Transformer} $
$ \newline C_L,C_W \text{- Line loss and Charging loss converted to Charger} $
$ \newline N_{i,min} \text{- Min no of Chargers in Charging Station i} $
$ \newline N_{i,max} \text{- Max no of Chargers in Charging Station i} $
$ \newline d_max \text{- Maximum Distance between charging station and fast charging demand point } $
$ \newline D_{ij} \text{- Linear Distance between charging stations i and j} $
$ \newline D_min \text{- Minimum Distance between charging stations} $

In [82]:
def calculate_F1(charging_station_coordinates_list,demand_by_service_area,r0=0.08, z=20, W=200*(10**4), q=5*(10**4), e=1.5*(10**4),mu=0.6, max_users=3,ni_min=10,ni_max=20,penalty_F1=1000):
    no_of_chargers=get_no_of_chargers( demand_by_service_area ,mu=0.6, max_users=3)
    fixed_value=0.0
    fixed_value=((r0)*((1+r0)**z))/(((1+r0)**z)-1)
    fcs={}
    total_value=0
    other_costs=0
    added_cost=0
    #print(fixed_value)
    for station in no_of_chargers.keys():
        fcs[station]=(W+(q*max(1,no_of_chargers[station]))+(e*((max(1,no_of_chargers[station]))**2)))
        fcs[station]+=(0.15*fcs[station])#ucs or operating cost
        #Doing Penalty for ni_min and ni_max corrections
        if no_of_chargers[station]<ni_min:
            other_costs+=abs((penalty_F1*(int(ni_min-no_of_chargers[station]))))
            fcs[station]+=abs((penalty_F1*(int(ni_min-no_of_chargers[station]))))
        elif no_of_chargers[station]>ni_max:
            other_costs+=abs((penalty_F1*(int(no_of_chargers[station]-ni_max))))
            fcs[station]+=abs((penalty_F1*(int(no_of_chargers[station]-ni_max)))) 
    #print(fcs)
    #print("***************************")
    #print(no_of_chargers,len(no_of_chargers.keys()))
    print("F1 OTHER COSTS=",other_costs)
    adder_text=""
    adder_text+="F1 OTHER COSTS="+str(other_costs)+"\n"
    return [sum(fcs.values()),adder_text]

        

In [83]:
def calculate_F2(point_region_dictionary,charging_station_coordinates_list,EV_demand,lambdaij=1.2,a=0.3,p=1,dmax=1.5,penalty_F2=100):
    f2={}
    other_costs=0
    station_to_demand_mapper_dict=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
    for station in station_to_demand_mapper_dict.keys():
        f2[station]=0.0
        for demand_point in station_to_demand_mapper_dict[station]:
            dist=np.sqrt(((station[0]-demand_point[0])**2)+((station[1]-demand_point[1])**2))
            f2[station]+=(lambdaij*EV_demand[demand_point]*dist)
            if (dist*lambdaij) > dmax:
                f2[station]+=penalty_F2*((dist*lambdaij)-dmax)
                other_costs+=penalty_F2*((dist*lambdaij)-dmax)
    print("F2 OTHER COSTS=",other_costs)
    adder_text=""
    adder_text+="F2 OTHER COSTS="+str(other_costs)+"\n"
    return [365*a*p*sum(f2.values()),adder_text]

In [84]:
def calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,demand_by_service_area, EV_demand,lambdaij=1.2,a=0.3,p=1, r0=0.08, z=20, W=200*(10**4), q=5*(10**4), e=1.5*(10**4),mu=0.6, max_users=3, ni_min=10, ni_max=20, dmax=1.5, Dmin=0.5,penalty_F1=1000,penalty_F2=100,penalty_total=100):
    adder_text=""
    F1_=calculate_F1(charging_station_coordinates_list,demand_by_service_area,r0, z, W, q, e,mu, max_users,ni_min,ni_max,penalty_F1)
    F2_=calculate_F2(point_region_dictionary,charging_station_coordinates_list,EV_demand,lambdaij,a,p,dmax,penalty_F2)
    adder_text+=F1_[1]+F2_[1]
    F1=F1_[0]
    F2=F2_[0]
    other_costs=0
    final_objective=F1+F2
    for stn1 in EV_demand.keys():
        for stn2 in EV_demand.keys():
            if stn1!=stn2:
                Dist=np.sqrt(((stn1[0]-stn2[0])**2)+((stn1[1]-stn2[1])**2))
                #print(Dist)
                if (lambdaij*Dist)<Dmin:
                    final_objective+=((Dmin-(lambdaij*Dist))*penalty_total)
                    other_costs+=((Dmin-(lambdaij*Dist))*penalty_total)
    print("TOTAL OTHER COSTS=",other_costs)
    adder_text+="TOTAL OTHER COSTS="+str(other_costs)+"\n"
    print("F1 ",F1)
    adder_text+="F1 "+str(F1)+"\n"
    print("F2",F2)
    adder_text+="F2 "+str(F2)+"\n"
    print("FINAL OBJECTIVE",final_objective)
    adder_text+="FINAL OBJECTIVE "+str(final_objective)+"\n"
    

    return [adder_text,final_objective]

In [85]:
def mutate(population,threshold=0.5,min_X=0,min_Y=0,max_X=200,max_Y=300,capper=50) :
    for p in range(len(population)):
        for g in range(len(population[p])):
            x=population[p][g][0]
            y=population[p][g][1]
            x_old=x
            y_old=y
            
            if random.random()<0.95:
                if random.random()<0.5:
                    x+=random.randint(1,capper)
                    if random.random()<0.5:
                        y+=random.randint(1,capper)
                    else:
                        y-=random.randint(1,capper)
                else:
                    x-=random.randint(1,capper)
                    if random.random()<0.5:
                        y+=random.randint(1,capper)
                    else:
                        y-=random.randint(1,capper)
                
                if not(x>max_X or x<min_X or y<min_Y or y>max_Y):
                    #print("MUTATED SUCCESSFULLY")
                    population[p][g][0]=x
                    population[p][g][1]=y
    return population.copy()
                

In [86]:
def crossover(population):
    population_status=[]
    temp_pop=population.copy()
    child_pop=[]
    while population_status!=[True]*len(population):
        p1=random.choice(temp_pop).copy()
        temp_pop.remove(p1)
        p2=random.choice(temp_pop).copy()
        temp_pop.remove(p2)
        c1=p1.copy()
        c2=p2.copy()
        for i in range(len(c1)//2):
            temp=c1[i].copy()
            c1[i]=c2[i].copy()
            c2[i]=temp.copy()
        child_pop.append(c1.copy())
        child_pop.append(c2.copy())
        population_status.append(True)
        population_status.append(True)
    return child_pop


In [87]:
def selection(parent_population,child_population,population_score,child_population_score): # Modify this later on with a better selection strategy(like roulette wheel)
    new_population=[]
    for i in range(len(parent_population)):
        p1=random.randint(0,len(parent_population)-1)
        p2=random.randint(0,len(parent_population)-1)
        c1=random.randint(0,len(child_population)-1)
        c2=random.randint(0,len(child_population)-1)
        
        p1_score=population_score[p1]
        p2_score=population_score[p2]
        c1_score=child_population_score[c1]
        c2_score=child_population_score[c2]
        
        if p1_score<=p2_score and p1_score<=c1_score and p1_score<=c2_score:
            new_population.append(population[p1].copy())
        elif p2_score<=c1_score and p2_score<=c2_score:
            new_population.append(population[p2].copy())
        elif c1_score<=c2_score:
            new_population.append(child_population[c1].copy())
        else:
            new_population.append(child_population[c2].copy())
    return new_population.copy()


In [88]:
lambdaij=1.2
a=0.3
p=1
r0=0.08
z=20
W=200*(10**4)
q=5*(10**4)
e=1.5*(10**4)
mu=0.6
max_users=3
ni_min=10
ni_max=20
dmax=1.5*10 #1.5
Dmin=1*10 #0.5
penalty_F1=100000
penalty_F2=100
penalty_total=100
def return_objective_value(pop,EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total):
    charging_station_coordinates=pop.copy()
    charging_station_coordinates_list=[]
    for point in charging_station_coordinates:
        charging_station_coordinates_list.append([float(point[0]),float(point[1])])
    points = np.array(charging_station_coordinates_list)
    vor = Voronoi(points.copy())
#     print("REGIONS \n",vor.regions)
#     print("VERTICES \n",vor.vertices)
#     print("POINT REGION \n",vor.point_region)
    region_to_vertex_mapping=region_to_vertex_mapper(vor.regions,vor.vertices)
    #print(region_to_vertex_mapping)
    point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list, region_to_vertex_mapping)
#     print("POINT REGION DICTIONARY",point_region_dictionary[(135.0, 175.0)])
    charging_station_to_region_no=charging_station_to_region_no_mapper(vor)
#     print("CHARGING STATION TO REGION NO\n",charging_station_to_region_no)
    charging_station_to_vertex=charging_station_vertex_mapper(vor)
#     print("CHARGING STATION TO VERTEX \n", charging_station_to_vertex)
    region_no_to_charging_station=region_no_to_charging_station_mapper(vor, charging_station_to_region_no, point_region_dictionary, region_to_vertex_mapping)
#     print("REGION NO TO CHARGING_STATION",region_no_to_charging_station)
    total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)
#    print("TOTAL DEMAND",total_demand)
#     xtemp=[]
#     ytemp=[]
#     text_=[]
#     for k,v in total_demand.items():
#         #print("K,V",k,v)
#         xtemp.append(k[0])
#         ytemp.append(k[1])
#         text_.append(v)
#     fig = plt.figure()
#     fig = voronoi_plot_2d(vor,point_size=30)
#     fig.set_figwidth(20)
#     fig.set_figheight(20)
#     plt.scatter(xcoords_centers,ycoords_centers,color='red',s=20)
#     plt.show()

#     plt.figure(figsize=(20,20))
#     plt.scatter(xtemp,ytemp)

#     for i in range(len(text_)):
#         plt.text(xtemp[i],ytemp[i],str(text_[i]),size=20)
    demand_point_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)
    #print("DEMAND TO STATION",demand_point_to_charging_station_mapper)
    station_to_demand=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
    obj=calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,total_demand, EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
    return obj
# import time
# no_of_members=10
# max_gen=100
# cur_gen=1
# nc=30
# population_tracker=[]
# import random
# population=[]
# for i in range(no_of_members):
#     temp=[]
#     for i in range(nc):
#         x=random.randint(1,max_X)
#         y=random.randint(1,max_Y)
#         temp.append([x,y])
#     population.append(temp)
# return_objective_value(population[0],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)

In [89]:
def return_best_solution(pop, pop_scores, best_solution_found,cur_gen):
    for i in range(len(pop_scores)):
        if best_solution_found[0]>pop_scores[i]:
            best_solution_found[0]=pop_scores[i]
            best_solution_found[1]=pop[i].copy()
            best_solution_found[2]=cur_gen
    return best_solution_found

### Initialization of parameters


In [90]:
import time
no_of_members=10
max_gen=1
cur_gen=1
nc=30
population_tracker=[]
import random
# population=[]
# for i in range(no_of_members):
#     temp=[]
#     for i in range(nc):
#         x=random.randint(1,max_X)
#         y=random.randint(1,max_Y)
#         temp.append([x,y])
#     population.append(temp)
# old_population=population.copy()


# definition of params

lambdaij=1.2
a=0.3
p=1
r0=0.08
z=20
W=200*(10**4)
q=5*(10**4)
e=1.5*(10**4)
mu=0.6
max_users=3
ni_min=10
ni_max=20
dmax=1.5*10 #1.5
Dmin=1*10 #0.5
penalty_F1=100000
penalty_F2=100
penalty_total=100
global_tracker=[]
best_solution_found=[10**20,[],0]


In [56]:
import time

for nc in [100]:
    no_of_members=10
    max_gen=100  #----@#$%@&@#%#@$^#^#@-------------------------------------@#$%@#$@#%@#^@$^@#%@^@^26
    cur_gen=1
    best_solution_found=[10**20,[],0]
    population_tracker_text=""
    start=time.time()
    print("************************************ NC=",nc);
    population_tracker_text+="************************************ NC="+str(nc)+"\n "
    population=[]
    for i in range(no_of_members):
        temp=[]
        for i in range(nc):
            x=random.randint(1,max_X)
            y=random.randint(1,max_Y)
            temp.append([x,y])
        population.append(temp)
    population_tracker=[]
    population_tracker.append(population.copy())
    obj_tracker=[]

    while cur_gen<=max_gen:
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = ",cur_gen)
        population_tracker_text=population_tracker_text+"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = "+str(cur_gen)+"\n "
        child_population=[]
        temp_population=crossover(population)
        child_population=mutate(temp_population,threshold=0.6,min_X=0,min_Y=0,max_X=max_X,max_Y=max_Y,capper=20)
        population_score={}
        child_population_score={}
        new_population_score={}
        for i in range(len(population)):
            print("PARENT POPULATION")
            population_tracker_text+="PARENT POPULATION "+str(cur_gen)+"\n"
            obj=return_objective_value(population[i],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
            population_score[i]=obj[1]
            population_tracker_text+=obj[0]
            obj_tracker.append((cur_gen,population[i],population_score[i],'P'))
        for j in range(len(child_population)):
            print("CHILD POPULATION")
            population_tracker_text+="CHILD POPULATION "+str(cur_gen)+"\n"
            obj=return_objective_value(child_population[j],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
            child_population_score[j]=obj[1]
            obj_tracker.append((cur_gen,child_population[j],obj,'C'))
            population_tracker_text+=obj[0]
        new_population=selection(population.copy(), child_population.copy(), population_score.copy(), child_population_score.copy()).copy()
        for k in range(len(new_population)):
            print("NEW POPULATION")
            population_tracker_text+="NEW POPULATION "+str(cur_gen)+"\n"
            obj=return_objective_value(new_population[k],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
            new_population_score[k]=obj[1]
            obj_tracker.append((cur_gen,new_population[k],obj,'N'))
            population_tracker_text+=obj[0]
        new_pop_tracker=return_best_solution(new_population.copy(),new_population_score,[10**20,[],0],cur_gen)
        population_tracker_text+="BEST SOLUTION FOUND IN THIS GENERATION"+str(new_pop_tracker)+"\n"
        best_solution_found=return_best_solution(new_population.copy(),new_population_score,best_solution_found,cur_gen)
        population=new_population.copy()
        cur_gen+=1
        population_tracker.append((cur_gen-1,new_population.copy(),new_population_score))
        #print("GENERATION=",cur_gen-1,"SCORE IS \n",new_population_score)
    
   
    population_tracker_text+="BEST SOLUTION FOUND UPTIL NOW"+str(best_solution_found)+"\n"
    end=time.time()
    global_tracker.append((nc,end-start,population_tracker.copy()))
    population_tracker_text+="################### TOTAL TIME TAKEN ################### = "+ str(end-start) +"\n"
    with open(str(nc)+" UPDATED GA EV LOG.txt", 'w') as f:
        f.write(population_tracker_text)


************************************ NC= 70
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION =  1
PARENT POPULATION
F1 OTHER COSTS= 15400000
F2 OTHER COSTS= 405533.45705299807
TOTAL OTHER COSTS= 0
F1  398971000.0
F2 52384552.798299454
FINAL OBJECTIVE 451355552.79829943
PARENT POPULATION
F1 OTHER COSTS= 24700000
F2 OTHER COSTS= 497666.3538101056
TOTAL OTHER COSTS= 0
F1  443386250.0
F2 63096010.76937628
FINAL OBJECTIVE 506482260.7693763
PARENT POPULATION
F1 OTHER COSTS= 15900000
F2 OTHER COSTS= 447943.3674439699
TOTAL OTHER COSTS= 0
F1  402121750.0
F2 57296587.0748845
FINAL OBJECTIVE 459418337.0748845
PARENT POPULATION
F1 OTHER COSTS= 11900000
F2 OTHER COSTS= 366134.311624396
TOTAL OTHER COSTS= 0
F1  388605500.0
F2 47646075.54109283
FINAL OBJECTIVE 436251575.5410928
PARENT POPULATION


KeyboardInterrupt: 

##### Particle Swarm Optimization

In [91]:
def best_swarm_position_initializer(pso_population,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total):
    g=10**10
    best_swarm_value=10**10
    best_swarm_position=-1
    for keys,values in pso_population.items():
        charging_station_coordinates=values.copy()
        charging_station_coordinates_list=[]
        for point in charging_station_coordinates:
            charging_station_coordinates_list.append([float(point[0]),float(point[1])])
        points = np.array(charging_station_coordinates_list)
        vor = Voronoi(points.copy())
        region_to_vertex_mapping=region_to_vertex_mapper(vor.regions,vor.vertices)
        point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list, region_to_vertex_mapping)
        charging_station_to_region_no=charging_station_to_region_no_mapper(vor)
        charging_station_to_vertex=charging_station_vertex_mapper(vor)
        region_no_to_charging_station=region_no_to_charging_station_mapper(vor, charging_station_to_region_no, point_region_dictionary, region_to_vertex_mapping)
        total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)
        demand_point_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)
        station_to_demand=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
        get_no_of_chargers(total_demand ,mu, max_users)
        obj=calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,total_demand, EV_demand,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
        if obj[1]<best_swarm_value:
            best_swarm_position=values.copy()
            best_swarm_value=obj[1]
    return (best_swarm_position.copy(),best_swarm_value)

def get_objective_value(particle,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total):

    charging_station_coordinates=particle.copy()
    charging_station_coordinates_list=[]
    for point in charging_station_coordinates:
        charging_station_coordinates_list.append([float(point[0]),float(point[1])])
    points = np.array(charging_station_coordinates_list)
    vor = Voronoi(points.copy())
    region_to_vertex_mapping=region_to_vertex_mapper(vor.regions,vor.vertices)
    point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list, region_to_vertex_mapping)
    charging_station_to_region_no=charging_station_to_region_no_mapper(vor)
    charging_station_to_vertex=charging_station_vertex_mapper(vor)
    region_no_to_charging_station=region_no_to_charging_station_mapper(vor, charging_station_to_region_no, point_region_dictionary, region_to_vertex_mapping)
    total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)
    demand_point_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)
    station_to_demand=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
    get_no_of_chargers(total_demand ,mu, max_users)
    obj=calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,total_demand, EV_demand,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
    return obj
    

In [92]:
lambdaij=1.2
a=0.3
p=1
r0=0.08
z=20
W=200*(10**4)
q=5*(10**4)
e=1.5*(10**4)
mu=0.6
max_users=3
ni_min=10
ni_max=20
dmax=1.5*10 #1.5
Dmin=1*10 #0.5
penalty_F1=100000
penalty_F2=100
penalty_total=100
global_tracker=[]
PSO_best_solution_found=[10**20,[],0]
PSO_population_tracker_text=""

In [94]:

import time

for nc in [30]:
    PSO_best_solution_found=[10**20,[],0]
    print("************************************ NC=",nc);
    PSO_population_tracker_text+="************************************ NC="+str(nc)+"\n "
    PSO_population_tracker_text=""
    start=time.time()
    no_of_particles=20
    max_iter=100  #!#$@#%#@%&@$^@$&@$^@$^@W$%@%#$@$%^%@$%@################^@#%@#$@
    cur_iter=1
    pso_population_tracker=[]
    particle_best_position={}
    import random
    pso_position={}
    for i in range(no_of_particles):
        temp=[]
        for j in range(nc):
            x=random.randint(1,max_X)
            y=random.randint(1,max_Y)
            temp.append([x,y])
        pso_position[i]=temp.copy()
        #print(pso_position)
    for keys,values in pso_position.items():
        particle_best_position[keys]=(values.copy(),get_objective_value(values.copy(),lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)[1])
        #print(particle_best_position[keys])
    best_swarm_position=best_swarm_position_initializer(pso_position,lambdaij,a,p,r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
    pso_velocity={}
    for i in range(no_of_particles):
        temp=[]
        for j in range(nc):
            x=random.randint(-(max_X-min_X),(max_X-min_X))
            y=random.randint(-(max_Y-min_Y),(max_Y-min_Y))
            temp.append([x,y])
        pso_velocity[i]=temp.copy()
    phig=random.uniform(1,3)
    phip=random.uniform(1,3)
    pso_population_tracker.append((pso_position.copy(),pso_velocity.copy()))  
    while cur_iter<=max_iter:
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = ",cur_iter)
        PSO_population_tracker_text+="%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = "+str(cur_iter)+"\n "
        PSO_score_list=[]
        for i in range(no_of_particles):
            for j in range(nc):
                rp=random.random()
                rg=random.random()
                w=random.random()
                pso_velocity[i][j][0]=int((w*pso_velocity[i][j][0])+(phip*rp*(particle_best_position[i][0][j][0]-pso_position[i][j][0]))+(phig*rg*(best_swarm_position[0][j][0]-pso_position[i][j][0])))
                pso_velocity[i][j][1]=int((w*pso_velocity[i][j][1])+(phip*rp*(particle_best_position[i][0][j][1]-pso_position[i][j][1]))+(phig*rg*(best_swarm_position[0][j][1]-pso_position[i][j][1])))
            new_coordinates=[]
            for j in range(nc):
                new_coordinates.append([pso_position[i][j][0]+pso_velocity[i][j][0],pso_position[i][j][1]+pso_velocity[i][j][1]])
            for j in range(nc):
                if not(new_coordinates[j][0]>=min_X and new_coordinates[j][0]<=max_X):
                    new_coordinates[j][0]=pso_position[i][j][0]
                if not(new_coordinates[j][1]>=min_Y and new_coordinates[j][1]<=max_Y):
                    new_coordinates[j][1]=pso_position[i][j][1]
            pso_position[i]=new_coordinates.copy()
            cur_value=get_objective_value(pso_position[i],lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
            PSO_score_list.append(cur_value[1])
            PSO_population_tracker_text+=str(cur_value[1])+"\n"
            PSO_population_tracker_text+=str(cur_value[0])+"\n"
            if cur_value[1]<particle_best_position[i][1]:
                particle_best_position[i]=(pso_position[i].copy(),cur_value[1])
            if cur_value[1]<best_swarm_position[1]:
                best_swarm_position=(pso_position[i].copy(),cur_value[1])
                PSO_best_solution_found=[cur_value[1],pso_position[i].copy(),cur_iter]
        cur_iter+=1            
        #print(best_swarm_position,cur_iter)
        pso_population_tracker.append((pso_position.copy(),pso_velocity.copy())) 
        new_pop_PSO_tracker=return_best_solution(list(pso_position.values()).copy(),PSO_score_list.copy(),[10**20,[],0],cur_iter)
        PSO_population_tracker_text+="BEST SOLUTION FOUND IN THIS GENERATION "+str(new_pop_PSO_tracker)+"\n"
    PSO_population_tracker_text+="BEST SOLUTION FOUND UPTIL NOW "+str(PSO_best_solution_found)+"\n"
    end=time.time()
    PSO_population_tracker_text+="################### TOTAL TIME TAKEN ################### = "+ str(end-start) +"\n"
    with open(str(nc)+" UPDATED PSO EV LOG.txt", 'w') as f:
        f.write(PSO_population_tracker_text)


************************************ NC= 30
F1 OTHER COSTS= 28500000
F2 OTHER COSTS= 1104506.0764205926
TOTAL OTHER COSTS= 0
F1  599728000.0
F2 133727431.39027752
FINAL OBJECTIVE 733455431.3902775
F1 OTHER COSTS= 30300000
F2 OTHER COSTS= 1241636.9799401532
TOTAL OTHER COSTS= 0
F1  585203750.0
F2 149688828.42581978
FINAL OBJECTIVE 734892578.4258198
F1 OTHER COSTS= 25000000
F2 OTHER COSTS= 1054423.356500678
TOTAL OTHER COSTS= 0
F1  563769250.0
F2 127957475.2819432
FINAL OBJECTIVE 691726725.2819432
F1 OTHER COSTS= 30300000
F2 OTHER COSTS= 1383717.3700029608


KeyboardInterrupt: 

In [61]:
particle_best_position[0]

499509179.9189682

In [ ]:
# End of Project File 
# Ram Ram